In [1]:
import sys
import os

import pandas as pd
import numpy as np

import pyarrow.parquet as pq

import datetime

In [2]:
root_dir = 'C:/Users/delst/OneDrive/Desktop/Code/Workspace/Crypto_V2'
sys.path.append(root_dir)

from A_Main.Configurations.setup_env import setup_environment
config = setup_environment(root_dir)

---

# **Collate Variable Data**

In [3]:
# RAW DATA
crypto_dir = os.listdir(os.path.join(config.raw_data, 'raw_data'))

# Load all raw data files, collate into a dataframe, and store in a dictionary
crypto_store = {}
for crypto in crypto_dir:
    crypto_dir_path = os.path.join(config.raw_data, 'raw_data', crypto)
    raw_file_list = os.listdir(crypto_dir_path)

    file_store = []
    for filename in raw_file_list:
        file_path = os.path.join(crypto_dir_path, filename)

        table = pq.read_table(file_path)
        
        df = table.to_pandas()
        df = df.drop(columns=['YEAR_AND_MONTH'])
        file_store.append(df)
    
    crypto_store[crypto] = pd.concat(file_store)

In [4]:
# Load 1st dataframe
print('Select Crypto File:', crypto_dir)

df = crypto_store[crypto_dir[0]]
df.head()

Select Crypto File: ['ADAUSDT', 'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'LTCUSDT', 'XRPUSDT']


,OPEN,HIGH,LOW,CLOSE,VOLUME,QUOTE_ASSET_VOLUME,NUMBER_OF_TRADES,TAKER_BUY_BASE_ASSET_VOLUME,TAKER_BUY_QUOTE_ASSET_VOLUME
OPEN_TIME,,,,,,,,,
1523937600000,0.25551,0.27000,0.25551,0.26666,268613.33,70602.884982,77,209583.50,55138.269746
1523937900000,0.26700,0.28800,0.26121,0.26696,1278878.30,342194.004025,567,410305.16,110163.574054
1523938200000,0.26698,0.28134,0.26389,0.26709,1372572.09,367736.101426,733,385452.37,103282.211192
1523938500000,0.26708,0.26869,0.26208,0.26331,977324.75,260132.114090,437,251076.69,66780.267625
1523938800000,0.26331,0.26436,0.26008,0.26290,887698.10,232282.163354,591,429640.80,112198.952292


---

# **Collate Label Data**

In [5]:
# LABEL DATA
label_dir = os.listdir(os.path.join(config.raw_data, 'label'))

# Load all label files, store in dictionary
label_store = {}
for crypto in label_dir:
    crypto_dir_path = os.path.join(config.raw_data, 'label', crypto)
    raw_file_list = os.listdir(crypto_dir_path)
    # print(raw_file_list)
    
    file_store = []
    for filename in raw_file_list:
        file_path = os.path.join(crypto_dir_path, filename)
        # print(file_path)

        table = pq.read_table(file_path)
        
        df = table.to_pandas()
        file_store.append(df)
    
    label_store[crypto] = pd.concat(file_store)

In [6]:
# Load 1st dataframe
print('Select Label File:', label_dir)

df = label_store[label_dir[0]]
df.head()

Select Label File: ['LABEL_ADAUSDT', 'LABEL_BNBUSDT', 'LABEL_BTCUSDT', 'LABEL_DOGEUSDT', 'LABEL_ETHUSDT', 'LABEL_LTCUSDT', 'LABEL_XRPUSDT']


,SYMBOL,LABEL
OPEN_TIME,,
1.526530e+12,ADAUSDT,0.366887
1.526533e+12,ADAUSDT,0.369106
1.526537e+12,ADAUSDT,0.414877
1.526540e+12,ADAUSDT,0.418307
1.526544e+12,ADAUSDT,0.402284


---

# **Format Datetimes**

### **Variables**

In [7]:
# Format datetime
for crypto in crypto_dir:
    crypto_store[crypto].index = pd.to_datetime(crypto_store[crypto].index, unit='ms')
    
# Resample to hourly and aggregate features
for crypto in crypto_dir:
    crypto_store[crypto] = crypto_store[crypto].resample('H').agg({
        'OPEN': 'first',
        'HIGH': 'max',
        'LOW': 'min',
        'CLOSE': 'last',
        'VOLUME': 'sum',
        'QUOTE_ASSET_VOLUME': 'sum',
        'NUMBER_OF_TRADES': 'sum',
        'TAKER_BUY_BASE_ASSET_VOLUME': 'sum',
        'TAKER_BUY_QUOTE_ASSET_VOLUME': 'sum'
    })

# Assign crypto label for later merging check
for crypto in crypto_dir:
    crypto_store[crypto]['crypto'] = crypto
    
dfs = [crypto_store[crypto] for crypto in crypto_dir]
dfs[0]

,OPEN,HIGH,LOW,CLOSE,VOLUME,QUOTE_ASSET_VOLUME,NUMBER_OF_TRADES,TAKER_BUY_BASE_ASSET_VOLUME,TAKER_BUY_QUOTE_ASSET_VOLUME,crypto
OPEN_TIME,,,,,,,,,,
2018-04-17 04:00:00,0.25551,0.28800,0.25551,0.26664,8143693.23,2.165077e+06,4421,2889823.93,7.671342e+05,ADAUSDT
2018-04-17 05:00:00,0.26660,0.27798,0.26010,0.26200,8317923.61,2.235633e+06,5149,3819527.71,1.025247e+06,ADAUSDT
2018-04-17 06:00:00,0.26221,0.26396,0.24800,0.25664,8420095.41,2.153964e+06,4566,3636154.31,9.302932e+05,ADAUSDT
2018-04-17 07:00:00,0.25662,0.26300,0.25489,0.25698,4686043.91,1.215621e+06,3283,1633201.94,4.234452e+05,ADAUSDT
2018-04-17 08:00:00,0.25636,0.25998,0.25229,0.25631,3510038.13,8.960957e+05,2408,1349216.91,3.444861e+05,ADAUSDT
...,...,...,...,...,...,...,...,...,...,...
2023-06-18 22:00:00,0.26060,0.26220,0.26000,0.26050,3392769.10,8.855565e+05,2132,1624311.70,4.241454e+05,ADAUSDT
2023-06-18 23:00:00,0.26050,0.26190,0.25880,0.26100,4979903.00,1.296618e+06,3829,2227235.00,5.798615e+05,ADAUSDT
2023-06-19 00:00:00,0.26100,0.26100,0.25710,0.25940,3447960.00,8.942890e+05,2904,1601383.00,4.149812e+05,ADAUSDT


### **Labels**

In [8]:
dfsl = [label_store[crypto] for crypto in label_dir]

for crypto in label_dir:
    label_store[crypto].index = pd.to_datetime(label_store[crypto].index, unit='ms')
dfsl[0]

,SYMBOL,LABEL
OPEN_TIME,,
2018-05-17 04:00:00,ADAUSDT,0.366887
2018-05-17 05:00:00,ADAUSDT,0.369106
2018-05-17 06:00:00,ADAUSDT,0.414877
2018-05-17 07:00:00,ADAUSDT,0.418307
2018-05-17 08:00:00,ADAUSDT,0.402284
...,...,...
2023-02-28 19:00:00,ADAUSDT,0.509147
2023-02-28 20:00:00,ADAUSDT,0.530118
2023-02-28 21:00:00,ADAUSDT,0.519972


---

# **Combine Variable and Lables**

In [9]:
# DATA COMBINE
merged_store = []
for df, df_label in zip(dfs, dfsl):
    merged_df = df.merge(df_label, left_index=True, right_index=True, how='inner')
    merged_store.append(merged_df)

[df.drop(columns=['crypto'], inplace=True) for df in merged_store]
[df.rename(columns={'LABEL': 'TARGET'}, inplace=True) for df in merged_store]

print(crypto_dir)

['ADAUSDT', 'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'LTCUSDT', 'XRPUSDT']


In [10]:
print(merged_store[0].SYMBOL[0])
merged_store[2]

ADAUSDT


,OPEN,HIGH,LOW,CLOSE,VOLUME,QUOTE_ASSET_VOLUME,NUMBER_OF_TRADES,TAKER_BUY_BASE_ASSET_VOLUME,TAKER_BUY_QUOTE_ASSET_VOLUME,SYMBOL,TARGET
OPEN_TIME,,,,,,,,,,,
2017-09-16 10:00:00,3753.29,3753.29,3470.66,3480.00,175.251826,6.277407e+05,778,58.976979,2.098699e+05,BTCUSDT,0.393885
2017-09-16 11:00:00,3490.00,3678.22,3490.00,3630.06,113.728279,4.099200e+05,475,68.053148,2.460002e+05,BTCUSDT,0.390215
2017-09-16 12:00:00,3630.06,3667.99,3565.47,3630.00,51.172541,1.852657e+05,347,23.788116,8.606734e+04,BTCUSDT,0.360434
2017-09-16 13:00:00,3610.00,3729.99,3531.01,3674.98,60.480745,2.194489e+05,413,36.841835,1.342355e+05,BTCUSDT,0.398650
2017-09-16 14:00:00,3674.98,3800.00,3630.00,3630.00,58.984953,2.189533e+05,664,14.245711,5.304050e+04,BTCUSDT,0.451718
...,...,...,...,...,...,...,...,...,...,...,...
2023-02-28 19:00:00,23481.63,23492.84,23205.06,23264.36,18237.246950,4.254090e+08,530354,8778.676070,2.047670e+08,BTCUSDT,0.448246
2023-02-28 20:00:00,23263.38,23343.55,23166.35,23272.05,15233.075630,3.543993e+08,423062,7509.378620,1.747199e+08,BTCUSDT,0.470401
2023-02-28 21:00:00,23270.59,23277.98,23020.97,23139.89,19752.939410,4.570602e+08,544036,9619.110940,2.225759e+08,BTCUSDT,0.468764


---

In [11]:
# A sample of the engineered data will now be saved 
df = df.head(1000)
path_to_save = os.path.join('C:/Users/delst/OneDrive/Desktop/Code/Workspace/Crypto_V2/Data/Samples', 'sample_eng.csv')
df.to_csv(path_to_save, index=True)